# SENG 550 Final Project
### Use the Amazon Appliances reviews dataset to develop a classifier or sentiment analyzer that can predict whether a given review is favorable or not

## Abstract

Our project uses the Amazon Appliances [reviews dataset](https://amazon-reviews-2023.github.io/) to develop a sentiment analyzer classification model. By combining star ratings and textual content, the model is trained to predict whether a review leans positively or negatively towards a product. This approach offers a way to quickly grasp the general sentiment of a product and assist shoppers in filtering through large volumes of product feedback more efficiently.

## Introduction

### Selected Problem

The problem aims to distinguish between favourable and unfavourable appliance reviews based on their text and accompanying reviews.

### Why is it Important?

Spending time reading review after review on a product becomes a burden. It is easy to misinterpret the mood behind a set of comments online which can easily lead to poor purchase decisions. A quick, automated sentiment indicator can ease the burden and establish a neutral decision making process.

### What have Others Done in this Space?

Researchers have performed [sentiment analysis](https://medium.com/@nafisaidris413/a-beginners-guide-for-product-review-sentiment-analysis-0de1f451167d) using Machine Learning and Natural Language Processing to automatically classify reviews as positive, negative, or neutral. Not only has sentiment analysis been applied to product reviews, it has also been applied to [social media](https://buffer.com/social-media-terms/sentiment-analysis) to determine how people perceive and talk about products and brands. This proves that data-driven classifiers are able to provide sentiment analysis scores for assist people in their daily lives, whether its to determine how their personal brand is viewed or how a to make an informed purchase through product review.

### Existing gaps?

Current solutions rely on product ratings to provide consumers with a sense of trust and quality to help them make purchasing decisions. This can be seen simply by going to any Amazon product and checking the reviews. Some reviews are informative with many positives about the product, though the product receives less than a rating of 5-stars, or the review is not informative whatsoever with a rating of 5-stars. Other times the reviews are clearly biased or the customer who leaves the review is disgruntled, leading to a 1- or 2-star rating. Using a combination of product star rating and textual review content, we are attempting to reveal patterns in product reviews that a rating alone might miss.

### Data Analysis Questions

1. Does text-based features add value beyond just a numerical rating?
2. Are there certain words which portray a stronger positve or negative sentiment?
3. How will adding text preprocessing impact accuracy?
4. Which models work best with this data?

### What is Proposed

We are proposing a text classification pipeline that merges a product's star rating and textual features.

### What are your Main Findings?

To determine customer opinion on various products within the Appliance category in Amazon's online store.

## Methodology

### Exploration of Data Features and Refinement of Feature Space

In this section, we focused on understanding the raw data collected from the collected [datasets](https://amazon-reviews-2023.github.io/) and transform them into a format suitable for model training. We begin by loading the Amazon Appliance reviews dataset and its corresponding metadata. We will explore the structure of the data, examine the distribution of fields we are interested in (like ratings), and assess the overall quality of the text reviews associated with the products. After we gain a thorough understanding, we apply a series of preprocessing techniques to clean and refine the text data. The goal here is to ultimately develop a set of features that can be fed into a machine learning model for sentiment classification.

#### Key Steps

1. **Loading the Data:**
We will loaf the `Appliances.jsonl` (reviews) and `meta_Appliances.jsonl` (metadata) using Apache Spark to avoid memory overload

2. **Initial Inspection and Basic Statistics:**
We will look at a few sample rows, check data types, count missing values, and examine distributions.

3. **Textual Data Exploration:**
We consider the nature of each review such as its length, the character composition, and common words. This should help guide our text cleaning decision.

4. **Data Cleaning:**
We clean the text by methods such as lowercasing the characters, removing punctuation, stripping leading and/or trailing whitespaces.

5. **Feature Transformation:**
We will use Spark Machine Learning's feature extraction tools to convert raw text into numeric features that are typically suitable for machine learning models.

#### Load the Data

The datasets are provided in `*.jsonl` format, which means each line is a separate JSON object representing a single review or product's metadata. We will use `SparkSession` to read the files which will handle the data in a distributed manner, effectively avoiding a potential kernel crash. Spark's lazy evaluation, transformations, and actions will manage memory usage of the large `.jsonl` files.

The two main data sources:
1. **Review File (`Appliances.jsonl`):**
Contains user-level reviews with fields such as `rating`, `title`, `text`, and `helpful_vote`.

2. **Metada File (`meta_Appliaces.jsonl`):**
Contains product-level information like `main_category`, `average_rating`, and `price`.

In [68]:
import os
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.sql.functions import col, isnan, when, count, expr, sum, size, lower, regexp_replace, min, avg, max
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType, MapType

In [61]:
# Start a Spark Session
spark = (
    SparkSession.builder
        .master("local")
        .appName("Amazon Review Analysis") 
        .config("spark.ui.showConsoleProgress", "false")
        .config("spark.executor.memory", "4g")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
)

# Only using columns needed for analysis for Reviews
reviews_schema = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("helpful_vote", IntegerType() , True),
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True)
])

# Only using columns needed for analysis for Metadata
meta_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", FloatType(), True),
    StructField("rating_number", IntegerType(), True),
    StructField("price", FloatType(), True),
    StructField("categories", ArrayType(StringType(), True), True),
    StructField("parent_asin", StringType(), True)
])

# Point to the location where the .jsonl files are
reviews_file = "./datasets/Appliances.jsonl"
meta_file = "./datasets/meta_Appliances.jsonl"

# Use the schema when reading the JSON file for Reviews
df_reviews = spark.read.schema(reviews_schema).json(reviews_file)

# Use the schema when reading the JSON file for Meta
df_meta = spark.read.schema(meta_schema).json(meta_file)

---

### Initial Inspection & Basic Statistics

First it is important to understand the size of the dataset we are dealing with and the distribution of ratings. Using Spark actions like `show()` and `count()` we determine some initial statistics about both datasets which will be helpful to visualize them. We also would like to know how many values in each column of the datasets are `null`, `None`, and `NaN`, in case we need to do some backfilling or should ignore those sets completely. We can also display a few rows of each dataset which makes sure that the datasets were loaded successfully.

#### Dataset Structure Inspection

For the structure of each dataset we will check the datasets dimensions, schema, and preview the data to understand each dataset.

In [62]:
# Printing Dimensions
print(f"Appliance Reviews dataset dimensions: {df_reviews.count()} rows, {len(df_reviews.columns)} columns")
print(f"Appliance Metadata dataset dimensions: {df_meta.count()} rows, {len(df_meta.columns)} columns")

Appliance Reviews dataset dimensions: 2128605 rows, 6 columns
Appliance Metadata dataset dimensions: 94327 rows, 7 columns


In [63]:
# Printing Schemas
print(f"Appliance Review Schema:")
df_reviews.printSchema()

print(f"\nAppliance Metadata Schema:")
df_meta.printSchema()

Appliance Review Schema:
root
 |-- rating: float (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- helpful_vote: integer (nullable = true)
 |-- asin: string (nullable = true)
 |-- parent_asin: string (nullable = true)


Appliance Metadata Schema:
root
 |-- main_category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- average_rating: float (nullable = true)
 |-- rating_number: integer (nullable = true)
 |-- price: float (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- parent_asin: string (nullable = true)



In [64]:
# Preview the Data
print(f"Appliance Review dataset preview:")
df_reviews.show(10, truncate=True)


Appliance Review dataset preview:
+------+--------------------+--------------------+------------+----------+-----------+
|rating|               title|                text|helpful_vote|      asin|parent_asin|
+------+--------------------+--------------------+------------+----------+-----------+
|   5.0|          Work great|work great. use a...|           0|B01N0TQ0OH| B01N0TQ0OH|
|   5.0|   excellent product|Little on the thi...|           0|B07DD2DMXB| B07DD37QPZ|
|   5.0|     Happy customer!|Quick delivery, f...|           0|B082W3Z9YK| B082W3Z9YK|
|   5.0|       Amazing value|I wasn't sure whe...|           0|B078W2BJY8| B078W2BJY8|
|   5.0|         Dryer parts|Easy to install g...|           0|B08C9LPCQV| B08C9LPCQV|
|   5.0|DO NOT purchase t...|After buying this...|           0|B08D6RFV6D| B099ZKQJHK|
|   2.0|They don't fit pr...|Not the best quality|           0|B001TH7GZA| B001TH7H0O|
|   5.0|          Five Stars|Part came quickly...|           0|B00AF7WZTM| B00AF7WZTM|
|   5.0| 

In [65]:
print(f"Appliance Metadata dataset preview:")
df_meta.show(10, truncate=True)

Appliance Metadata dataset preview:
+--------------------+--------------------+--------------+-------------+-----+--------------------+-----------+
|       main_category|               title|average_rating|rating_number|price|          categories|parent_asin|
+--------------------+--------------------+--------------+-------------+-----+--------------------+-----------+
|Industrial & Scie...|ROVSUN Ice Maker ...|           3.7|           61| NULL|[Appliances, Refr...| B08Z743RRD|
|Tools & Home Impr...|HANSGO Egg Holder...|           4.2|           75| NULL|[Appliances, Part...| B097BQDGHJ|
|Tools & Home Impr...|Clothes Dryer Dru...|           3.5|           18| NULL|[Appliances, Part...| B00IN9AGAE|
|Tools & Home Impr...|154567702 Dishwas...|           4.5|           26| NULL|[Appliances, Part...| B0C7K98JZS|
|Tools & Home Impr...|Whirlpool W109185...|           3.8|           12|25.07|[Appliances, Part...| B07QZHQTVJ|
|Tools & Home Impr...|1841N030 - Brown ...|           4.5|          

#### Missing Values Inspection

Missing values is one of the most common headaches in datasets. We need to check for them in both datasets to ensure that we can confidently use the data, else we have to consider backfilling the missing data or not using it at all.

In [46]:
reviews_float_columns = [c for c in df_reviews.columns if df_reviews.schema[c].dataType.simpleString() == "float"]
reviews_string_columns = [c for c in df_reviews.columns if df_reviews.schema[c].dataType.simpleString() == "string"]
reviews_integer_columns = [c for c in df_reviews.columns if df_reviews.schema[c].dataType.simpleString() == "int"]

reviews_float_null = df_reviews.select([sum(col(c).isNull().cast("int")).alias(c) for c in reviews_float_columns])
reviews_string_null = df_reviews.select([sum((col(c).isNull() | (col(c) == "")).cast("int")).alias(c) for c in reviews_string_columns])
reviews_integer_null = df_reviews.select([sum(col(c).isNull().cast("int")).alias(c) for c in reviews_integer_columns])

df_reviews_null = (
    reviews_float_null.select(*(col(c).alias(f"{c}") for c in reviews_float_null.columns))
    .crossJoin(reviews_string_null.select(*(col(c).alias(f"{c}") for c in reviews_string_null.columns)))
    .crossJoin(reviews_integer_null.select(*(col(c).alias(f"{c}") for c in reviews_integer_null.columns)))
)

print(f"Appliance Reviews Counted Missing Values per Column")
df_reviews_null.show()

Appliance Reviews Counted Missing Values per Column
+------+-----+----+----+-----------+------------+
|rating|title|text|asin|parent_asin|helpful_vote|
+------+-----+----+----+-----------+------------+
|     0|    0|  95|   0|          0|           0|
+------+-----+----+----+-----------+------------+



In [47]:
# Define lists of column names for each type
meta_float_columns = [c for c in df_meta.columns if df_meta.schema[c].dataType.simpleString() == "float"]
meta_string_columns = [c for c in df_meta.columns if df_meta.schema[c].dataType.simpleString() == "string"]
meta_integer_columns = [c for c in df_meta.columns if df_meta.schema[c].dataType.simpleString() == "int"]
meta_array_columns = [c for c in df_meta.columns if df_meta.schema[c].dataType.simpleString().startswith("array")]

# Compute NULL counts for float columns
meta_float_null = df_meta.select([sum(col(c).isNull().cast("int")).alias(c) for c in meta_float_columns])

# Compute NULL or empty counts for string columns
meta_string_null = df_meta.select([sum((col(c).isNull() | (col(c) == "")).cast("int")).alias(c) for c in meta_string_columns])

# Compute NULL counts for integer columns
meta_integer_null = df_meta.select([sum(col(c).isNull().cast("int")).alias(c) for c in meta_integer_columns])

# Compute NULL or empty counts for array columns
meta_array_null = df_meta.select([sum((col(c).isNull() | expr(f"exists({c}, x -> x == '')")).cast("int")).alias(c)for c in meta_array_columns])

# Combine all the results into one DataFrame
df_meta_null = (
    meta_float_null.select(*(col(c).alias(f"{c}") for c in meta_float_null.columns))
    .crossJoin(meta_string_null.select(*(col(c).alias(f"{c}") for c in meta_string_null.columns)))
    .crossJoin(meta_integer_null.select(*(col(c).alias(f"{c}") for c in meta_integer_null.columns)))
    .crossJoin(meta_array_null.select(*(col(c).alias(f"{c}") for c in meta_array_null.columns)))
)

# Print the results
print(f"Appliance Metadata Counted Missing Values per Column")
df_meta_null.show()

Appliance Metadata Counted Missing Values per Column


+--------------+-----+-------------+-----+-----------+-------------+----------+
|average_rating|price|main_category|title|parent_asin|rating_number|categories|
+--------------+-----+-------------+-----+-----------+-------------+----------+
|             0|47601|         4676|    9|          0|            0|         0|
+--------------+-----+-------------+-----+-----------+-------------+----------+



#### Duplicates

Catching if there is duplicate data is important so that we do not have skewed data. All the data should be unique. Duplicate data will be handled during data cleanup.

In [48]:
print(f"Appliance Reviews Duplicate Data: {df_reviews.count() - df_reviews.distinct().count()}")
print(f"Appliance Metadate Duplicate Data: {df_meta.count() - df_meta.distinct().count()}")

Appliance Reviews Duplicate Data: 29492
Appliance Metadate Duplicate Data: 0


#### Statistical Data

We examine the statistical summaries for both the Reviews and Metadata. This helps us spot outliers and check if there are unexpected ranges that we have to look out for.

In [50]:
print(f"Appliance Reviews Statistical Summary")
df_reviews.describe().show()

Appliance Reviews Statistical Summary
+-------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+
|summary|            rating|               title|                text|      helpful_vote|                asin|         parent_asin|
+-------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+
|  count|           2128605|             2128605|             2128605|           2128605|             2128605|             2128605|
|   mean| 4.221502345432807|                 NaN|1.0294495574587156E9|0.9288867591685634|1.5550635848728814E9|1.5550635848728814E9|
| stddev|1.3808261737697332|                 NaN|1.064196457532015...|12.526794316769578|1.4548141211071746E9|1.4548141211071746E9|
|    min|               1.0|                   !|                    |                 0|          0967805929|          0967805929|
|    max|               5.0|🧊🥶 AMAZING

In [51]:
print(f"Appliance Metadata Statistical Summary")
df_meta.describe().show()

Appliance Metadata Statistical Summary
+-------+--------------+--------------------+------------------+------------------+------------------+--------------------+
|summary| main_category|               title|    average_rating|     rating_number|             price|         parent_asin|
+-------+--------------+--------------------+------------------+------------------+------------------+--------------------+
|  count|         89651|               94327|             94327|             94327|             46726|               94327|
|   mean|          NULL|  1.1113368793875E10| 4.118858857941276|136.36790102515715|  86.4799539034291|4.0776468745555553E9|
| stddev|          NULL|3.142601103602334...|0.8640397544170944| 977.5160999553561|325.31839674168526| 3.745278366512328E9|
|    min|AMAZON FASHION|                    |               1.0|                 1|              0.01|          0967805929|
|    max|   Video Games|𝟮𝟬𝟮𝟯𝙪𝙥𝙜𝙧?...|               5.0|             90203|          21095.62

---

### Textual Data Exploration

The main predictive feature of the model will likely be the `text` column in the Appliance Review dataset. It is important that we are able to understand its quality. We find answers to questions such as are the reviews too short or lone? Do they contain descriptive terms or just a few words? We also need to consider if there is text in different languages other than english.

What we will do is start by analyzing the length of the reviews in terms of word count. This will help guide us in the direction we want. If the text is too short, maybe we need to rely more on ratings or metadata. If text is rich, a text-based sentiment analysis may just work well.

In [ ]:
df_reviews = df_reviews.fillna({"text": ""})
df_reviews = df_reviews.withColumn("text", col("text").cast(StringType()))

tokenizer_reviews = Tokenizer(inputCol="text", outputCol="words_raw")
df_tokenizer_reviews = tokenizer_reviews.transform(df_reviews)

df_tokenizer_reviews = df_tokenizer_reviews.withColumn("character_length", size(col("words_raw")))

print(f"Appliance Reviews with Raw Words and Character Length included")
df_tokenizer_reviews.show(10, truncate=True)

print(f"\nMinimum, Average, and Maximum character length of the Appliance Reviews")
df_tokenizer_reviews.select(min("character_length"), avg("character_length"), max("character_length")).show()

Appliance Reviews with Raw Words and Character Length included
+------+--------------------+--------------------+------------+----------+-----------+--------------------+----------------+
|rating|               title|                text|helpful_vote|      asin|parent_asin|           words_raw|character_length|
+------+--------------------+--------------------+------------+----------+-----------+--------------------+----------------+
|   5.0|          Work great|work great. use a...|           0|B01N0TQ0OH| B01N0TQ0OH|[work, great., us...|               8|
|   5.0|   excellent product|Little on the thi...|           0|B07DD2DMXB| B07DD37QPZ|[little, on, the,...|               5|
|   5.0|     Happy customer!|Quick delivery, f...|           0|B082W3Z9YK| B082W3Z9YK|[quick, delivery,...|               5|
|   5.0|       Amazing value|I wasn't sure whe...|           0|B078W2BJY8| B078W2BJY8|[i, wasn't, sure,...|              57|
|   5.0|         Dryer parts|Easy to install g...|           0

---

### Data Cleaning

In [60]:
spark.stop()